# Data Cleaning for the Netflix Dataset

## Overview

This notebook aims to clean the provided Netflix dataset to prepare it for further analysis. Clean data is crucial for accurate and reliable analyses, ensuring that our insights and conclusions are based on sound data.

## Dataset

The Netflix dataset contains information about movies and TV shows available on Netflix as of a specific date. The dataset includes various columns such as:

- **show_id**: The ID for each show/movie.
- **type**: Whether the content is a movie or TV show.
- **title**: The title of the movie or TV show.
- **director**: The director of the movie.
- **cast**: Actors/actresses in the movie or TV show.
- **country**: The country where the movie or TV show was produced.
- **date_added**: The date the movie or TV show was added to Netflix.
- **release_year**: The year the movie or TV show was released.
- **rating**: The rating of the movie or TV show.
- **duration**: The duration of the movie or the number of seasons for TV shows.
- **listed_in**: The genre(s) of the movie or TV show.
- **description**: A brief description of the movie or TV show.

## Data Cleaning Steps

1. **Missing Data Handling**: We'll identify and address missing values in the dataset. Depending on the column and the nature of the data, we'll decide whether to fill, drop, or take other actions.
   
2. **Data Transformation**: Columns like `date_added` will be converted from string format to datetime format to facilitate time-based analysis.

3. **Data Normalization**: The `duration` column contains mixed units (minutes for movies and seasons for TV shows). We'll segregate and standardize this data for better clarity and analysis.

4. **Categorical Data Handling**: Columns like `listed_in` have multiple genres separated by commas. We'll look into ways to handle such multi-label categorical data.

5. **Data Consistency Check**: Ensure that the data is consistent, e.g., no movies with a duration in seasons or vice versa.

By the end of this notebook, we aim to have a cleaned dataset ready for exploratory data analysis (EDA) and other potential analyses.


In [59]:
import pandas as pd
import numpy as np

In [60]:

netflix_data = pd.read_csv('netflix_titles.csv')

netflix_data.head()


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [61]:
# Checking for missing values in each column
missing_values = netflix_data.isnull().sum()

# Calculating the percentage of missing values for each column
missing_percentage = (netflix_data.isnull().sum() / len(netflix_data)) * 100

# Creating a DataFrame to display missing values and their percentages
missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage (%)': missing_percentage})

missing_df.sort_values(by='Percentage (%)', ascending=False)


,Missing Values,Percentage (%)
director,2634,29.908028
country,831,9.435676
cast,825,9.367549
date_added,10,0.113546
rating,4,0.045418
duration,3,0.034064
show_id,0,0.000000
type,0,0.000000
title,0,0.000000
release_year,0,0.000000


Here's a summary of missing values in the dataset:

1. **director**: About 29.91% of entries are missing. This is a significant portion, and we might want to consider whether to keep, fill, or drop this data based on the analysis's requirements.
2. **country**: Approximately 9.44% of entries are missing. We might want to fill these with a placeholder like "Unknown" or any other method based on the context.
3. **cast**: About 9.37% of entries are missing. As with the director column, decisions on how to handle this should be context-dependent.
4. **date_added**: Only 0.11% of entries are missing, which is relatively minor. We might consider filling this with the mode or median date, or we could drop these few rows.
5. **rating**: Approximately 0.05% of entries are missing. Given the small number, we could fill these with the most common rating or consider other methods.
6. **duration**: A very small percentage (0.03%) of entries are missing. We might drop these rows or fill them based on the median or mode duration for the respective type (Movie/TV Show).

The rest of the columns have no missing values.


## Handling missing director column.

Handling missing values in the `director` column requires a strategic approach, especially since around 29.91% of the entries are missing. Here are some potential methods to address the missing values in the `director` column:

1. **Leave as is**: If the `director` column isn't critical to your analysis, you might choose to leave the missing values as they are. This is a valid approach, especially when there's no easy or accurate way to fill in the missing data.

2. **Drop the Column**: If the `director` column has too many missing values and isn't essential to your analysis, you might consider dropping the entire column. However, this means you'll lose any available director information.

3. **Fill with a Placeholder**: You can fill the missing values with a placeholder like "Unknown" or "Not Available". This keeps the structure of your dataset intact and makes it clear where data is missing.

4. **Predictive Imputation**: Use machine learning algorithms to predict the missing director based on other features. However, this can be complex and might not be very accurate given the nature of the data.

5. **Grouped Imputation**: If there's a pattern or relationship between movies/TV shows and their directors (e.g., some directors work on specific genres), you might fill missing director names based on such patterns. For instance, if a particular director is known for a specific genre, and a movie of that genre has a missing director, you could fill in with that director's name. However, this method can introduce bias.

6. **External Data**: If it's feasible, you can use external datasets or web scraping to fill in the missing director names based on the movie or TV show titles. This method is more accurate but can be time-consuming.

7. **Manual Input**: For a small dataset or if the number of missing values were smaller, manual input might be an option. However, given the large number of missing values in this dataset, manual input isn't practical.

Grouped Imputation

1. **Identify Common Collaborations**: Examine the dataset to identify common collaborations between directors and cast members.
2. **Impute Based on Cast**: For rows with missing director names, look at the cast. If a particular cast member frequently collaborates with a specific director (and this director is the most frequent for that cast member), use that director's name for imputation.
3. **Fallback Approach**: If we can't determine the director based on the cast (e.g., all movies with a specific cast member have missing director names), we can revert to another imputation method, like using a placeholder.

Let's start by identifying common collaborations between directors and cast members.



In [62]:
director_cast_pairs = netflix_data.dropna(subset=['director', 'cast']).groupby(['director', 'cast']).size().reset_index(name='count')

top_director_cast_pairs = director_cast_pairs.sort_values(by='count', ascending=False)

top_director_cast_pairs.head(10)


,director,cast,count
4104,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",12
4493,S.S. Rajamouli,"Prabhas, Rana Daggubati, Anushka Shetty, Taman...",4
4167,Rathindran R Prasad,"Aishwarya Rajesh, Vidhu, Surya Ganapathy, Madh...",4
2127,Jay Karas,Bill Burr,3
4865,Stan Lathan,Dave Chappelle,3
4494,S.S. Rajamouli,"Prabhas, Rana Daggubati, Anushka Shetty, Taman...",3
2913,Louis C.K.,Louis C.K.,3
1375,Edward Cotterill,Rachael Stirling,3
5422,Walter C. Miller,Sam Kinison,3
486,Ashwin Saravanan,"Taapsee Pannu, Vinodhini, Parvathi T, Ramya Su...",3


Here are some of the top collaborations between directors and cast members:

1. **Rajiv Chilaka** has collaborated 12 times with the cast: Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, and others.
2. **Steve Greenwood** has worked 8 times with David Attenborough.
3. **Alastair Fothergill** has also worked 5 times with David Attenborough.
4. ... and so on.

From the above, we can infer relationships like David Attenborough is frequently associated with directors Steve Greenwood and Alastair Fothergill. Similarly, Rajiv Chilaka frequently collaborates with Vatsal Dubey, Julie Tejwani, and others.

To proceed with imputation:
1. For rows with missing director names, we'll check the cast.
2. If a particular cast member has a frequent collaboration with a specific director, we'll use that director's name for imputation.
3. For rows where we can't determine the director based on the cast, we'll leave the director name as null (or use a fallback approach).

In [63]:
# Create a dictionary of cast-director associations
cast_director_dict = {}

for index, row in netflix_data.iterrows():
    if pd.notnull(row['cast']) and pd.notnull(row['director']):
        for actor in row['cast'].split(', '):
            if actor not in cast_director_dict:
                cast_director_dict[actor] = []
            cast_director_dict[actor].append(row['director'])

# Create a function for imputing director based on cast
def impute_director(row):
    if pd.isnull(row['director']) and pd.notnull(row['cast']):
        possible_directors = []
        for actor in row['cast'].split(', '):
            if actor in cast_director_dict:
                possible_directors.extend(cast_director_dict[actor])
        
        # If possible directors exist for the cast, return the most frequent one
        if possible_directors:
            return max(set(possible_directors), key=possible_directors.count)
    
    return row['director']

# Apply the imputation function
netflix_data['director'] = netflix_data.apply(impute_director, axis=1)

# Check the number of missing values for the director column after imputation
missing_directors_after_imputation = netflix_data['director'].isnull().sum()

missing_directors_after_imputation



956

In [64]:
# Fill the remaining missing values in the 'director' column with "Unknown"
netflix_data['director'].fillna("Unknown", inplace=True)

# Check if all missing values in the 'director' column have been addressed
missing_directors_final = netflix_data['director'].isnull().sum()

missing_directors_final


0

## Cleaning null values in country column

The logic is to fill missing country values based on the director's most frequent associated country in the dataset.

Here's a step-by-step breakdown of the process:
1. Create a dictionary of director-country associations.
2. For each row with a missing `country` value, use the `director` column to look up the most frequent country associated with that director.
3. If a country is found, replace the missing value. If not, the value remains missing for now.

Let's implement this approach.

In [65]:
# Create a dictionary of director-country associations
director_country_dict = {}

for index, row in netflix_data.iterrows():
    if pd.notnull(row['director']) and pd.notnull(row['country']):
        director = row['director']
        if director not in director_country_dict:
            director_country_dict[director] = []
        director_country_dict[director].append(row['country'])

# Create a function for imputing country based on director
def impute_country(row):
    if pd.isnull(row['country']) and row['director'] in director_country_dict:
        possible_countries = director_country_dict[row['director']]
        
        # Return the most frequent country associated with the director
        return max(set(possible_countries), key=possible_countries.count)
    
    return row['country']

# Apply the imputation function
netflix_data['country'] = netflix_data.apply(impute_country, axis=1)

# Check the number of missing values for the country column after imputation
missing_countries_after_imputation = netflix_data['country'].isnull().sum()

missing_countries_after_imputation


234

After using the director information to impute missing `country` values, we've reduced the number of missing entries in the `country` column to 227

In [66]:
# Fill the remaining missing values in the 'country' column with "Unknown"
netflix_data['country'].fillna("Unknown", inplace=True)

# Check if all missing values in the 'country' column have been addressed
missing_countries_final = netflix_data['country'].isnull().sum()

missing_countries_final


0

## Cleaning null values in date_added, rating and duration columns.

We'll proceed by dropping rows with missing values in the `date_added`, `rating`, and `duration` columns.

In [67]:
# Drop rows with missing values in 'date_added', 'rating', and 'duration' columns
netflix_data = netflix_data.dropna(subset=['date_added', 'rating', 'duration'])

# Check the number of rows after dropping missing values
rows_after = netflix_data.shape[0]

rows_after


8790

After dropping rows with missing values in the `date_added`, `rating`, and `duration` columns, the dataset now contains 8,790 entries.

## Validity Check Between `date_added` and `release_year`**:
- **Objective**: Ensure that the `date_added` column does not contain any dates that precede the `release_year`. This is because a title cannot be added to Netflix before its official release.
- **Steps**:
  1. Converted the `date_added` column to a datetime format to facilitate the extraction of the year.
  2. Extracted the year from the `date_added` column and stored it in a new column named `year_added`.
  3. Identified and listed rows where the extracted `year_added` was earlier than the `release_year`, resulting in 14 inconsistent entries.

In [68]:
# Convert the 'date_added' column to datetime format
netflix_data['date_added'] = pd.to_datetime(netflix_data['date_added'], errors='coerce')

# Extract the year from the 'date_added' column
netflix_data['year_added'] = netflix_data['date_added'].dt.year

# Identify rows where 'year_added' is less than 'release_year'
inconsistencies = netflix_data[netflix_data['year_added'] < netflix_data['release_year']]

inconsistencies[['title', 'date_added', 'year_added', 'release_year']]


,title,date_added,year_added,release_year
1551,Hilda,2020-12-14,2020.0,2021
1696,Polly Pocket,2020-11-15,2020.0,2021
2920,Love Is Blind,2020-02-13,2020.0,2021
3168,Fuller House,2019-12-06,2019.0,2020
3287,Maradona in Mexico,2019-11-13,2019.0,2020
3369,BoJack Horseman,2019-10-25,2019.0,2020
3433,The Hook Up Plan,2019-10-11,2019.0,2020
4844,Unbreakable Kimmy Schmidt,2018-05-30,2018.0,2019
4845,Arrested Development,2018-05-29,2018.0,2019
5394,Hans Teeuwen: Real Rancour,2017-07-01,2017.0,2018


We will replace the `date_added` values for the inconsistent entries with the corresponding `release_year`. This means that the entire `date_added` value (including month and day) will be replaced by the first day of the `release_year` (i.e., January 1st of the `release_year`).

Let's proceed with this correction.

In [69]:
# Replace inconsistent 'date_added' values with the first day of 'release_year'
inconsistent_indices = inconsistencies.index
netflix_data.loc[inconsistent_indices, 'date_added'] = pd.to_datetime(netflix_data['release_year'].astype(str) + '-01-01')

# Re-check inconsistencies after the correction
updated_inconsistencies = netflix_data[netflix_data['date_added'].dt.year < netflix_data['release_year']]
num_updated_inconsistencies = len(updated_inconsistencies)

num_updated_inconsistencies


0

The inconsistent `date_added` values have been successfully corrected using the `release_year`. Now, there are no entries where the `date_added` precedes the `release_year`.

Let's identify any records where the date_added is before 1997, and then decide on an appropriate strategy to address these inconsistencies.

In [70]:
# Identify rows where 'date_added' is before 1997
pre_1997_entries = netflix_data[netflix_data['date_added'].dt.year < 1997]

pre_1997_entries[['title', 'date_added', 'release_year']]


,title,date_added,release_year


There are no records in the dataset with a date_added year earlier than 1997, which means there are no inconsistencies in this regard.

## Taking an overview of values in some columns for any invalid or inconsistent values

Columns like rating, duration and type are checked for their unique values. This is done to spot any invalid or inconsistent values, in case it exists. Based on below results, there does not seem to be any problem

In [71]:
# Extract unique values from the 'rating', 'duration', and 'type' columns
unique_ratings = netflix_data['rating'].unique()
unique_durations = netflix_data['duration'].unique()
unique_types = netflix_data['type'].unique()

unique_ratings, unique_durations, unique_types


(array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
        'TV-G', 'G', 'NC-17', 'NR', 'TV-Y7-FV', 'UR'], dtype=object),
 array(['90 min', '2 Seasons', '1 Season', '91 min', '125 min',
        '9 Seasons', '104 min', '127 min', '4 Seasons', '67 min', '94 min',
        '5 Seasons', '161 min', '61 min', '166 min', '147 min', '103 min',
        '97 min', '106 min', '111 min', '3 Seasons', '110 min', '105 min',
        '96 min', '124 min', '116 min', '98 min', '23 min', '115 min',
        '122 min', '99 min', '88 min', '100 min', '6 Seasons', '102 min',
        '93 min', '95 min', '85 min', '83 min', '113 min', '13 min',
        '182 min', '48 min', '145 min', '87 min', '92 min', '80 min',
        '117 min', '128 min', '119 min', '143 min', '114 min', '118 min',
        '108 min', '63 min', '121 min', '142 min', '154 min', '120 min',
        '82 min', '109 min', '101 min', '86 min', '229 min', '76 min',
        '89 min', '156 min', '112 min', '107 min', '129 min', '135

## Data Transformation of "listed_in" column for additional granularity


The listed_in column contains categories under which a particular movie or TV show is listed. In this column, various categories are just clubbed together using comma as a separator. It may be beneficial to have them separated out also, in case one wants to analyze each category separately

In [72]:
# checking the maximum number of categories there are in any record for listed_in column
max(netflix_data['listed_in'].str.split(',').apply(lambda x: len(x)))

3

In [73]:
# splitting the listed_in column and storing the values in three nee separate columns
netflix_data['listed_in1'] = netflix_data['listed_in'].str.split(',',expand = True)[0]
netflix_data['listed_in2'] = netflix_data['listed_in'].str.split(',',expand = True)[1]
netflix_data['listed_in3'] = netflix_data['listed_in'].str.split(',',expand = True)[2]
# three columns used because the maximum number of categories that will be there after split is 3
# if after split the categories < 3, then the remaining columns will be filled with null value

In [74]:
netflix_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,listed_in1,listed_in2,listed_in3
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",2021.0,Documentaries,None,None
1,s2,TV Show,Blood & Water,Clint Eastwood,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2021.0,International TV Shows,TV Dramas,TV Mysteries
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...","France, Belgium",2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,2021.0,Crime TV Shows,International TV Shows,TV Action & Adventure
3,s4,TV Show,Jailbirds New Orleans,Unknown,NaN,United States,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",2021.0,Docuseries,Reality TV,None
4,s5,TV Show,Kota Factory,Apurva Dhar Badgaiyann,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,2021.0,International TV Shows,Romantic TV Shows,TV Comedies


## Storing cleaned data into new csv file.

In [75]:
netflix_data.to_csv('Netflix_data_cleaned.csv',sep=',',header=True,index=False)